## Importing Basic Librarys Required 

In [4]:
import tensorflow as tf
import cv2
import os
import time
import random
import matplotlib.pyplot as plt
import numpy as np # TO load matrix to array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Flatten , Conv2D , MaxPooling2D , BatchNormalization ,Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard

#print(tf.__version__)  #we are Using 1.9.0 
#print(cv2.__version__) #we are Using 3.4.1 


## Models Use


## Creating Data Set For Traning 

In [5]:
NAME="FACE-RECO-{}".format(int(time.time()))

tensorboard= TensorBoard (log_dir='log/{}'.format(NAME))

DATADIR = '/home/abhishek/ml/machinelearning-/data' #Original Path to data
CATEGORIES = ["bill gates","donald trump","steve jobs"] #This includes Folders in which Images are kept
IMG_SIZE= 300 # Variable Use to Resize Image For training 

for category in CATEGORIES:
    path = os.path.join(DATADIR,category)
 #   print(path)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        img_array_resize= cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
       # plt.imshow(img_array_resize,cmap="gray") #To print image
        #plt.show() # To show ploted Image
    
#print(img_array_resize.shape)

## Creating Traing Data

In [6]:
training_data = []

def create_training_data():
    for category in CATEGORIES:
       #  print(category)
        path = os.path.join(DATADIR,category) #create path to Training Image
        # print(path)
        class_num = CATEGORIES.index(category) #get the Classification (0 or 1)  0=bill gates and 1= trump
        # print(class_num)
 #   print(path)
    for img in os.listdir(path):  # this will itrate over each image in data set as per the CATEGORIES
            try :
                img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                new_array_resize= cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array_resize,class_num])
            except Exception as e: # Too Pass as Os or path errors which is not affecting any traing data 
                pass
                plt.imshow(new_array_resize,cmap="gray")
                plt.show() # To show ploted Image

#print(img_array_resize.shape)
create_training_data()
#print(len(training_data)) #This will print the length of that i.e 23 
#print(training_data) 
# This will print the 300x300 matrix of the image 
random.shuffle(training_data) #Shuffling Data so all images get mixed and The network doesn't make any pattern to memorize the data 
#for sample in training_data:  #For Checking Data in 
  #  print(sample[1])

## Making Model 

In [7]:
X=[]
y=[]

for features, label in training_data:
    X.append(features)
    y.append(label)
#print(y)
y=to_categorical(y)
X=np.array(X)
X=X/255.0
X= np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1)
#print(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

## Creating Neural Networks 

In [10]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.20))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.20))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.20))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.20))

model.add(Dense(3))
model.add(Activation('sigmoid'))

#model.summary()

In [1]:
## EarlyStopping is use to Interrupt training when the Validation Loss isn't Decreasing anymore 


In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#early_stop = EarlyStopping(monitor='val_loss', patience=100, mode='auto')
#model.fit(X_train, y_train, batch_size=3, epochs=5, validation_data=(X_val, y_val))
model.fit(X_train, y_train, batch_size=6, epochs=5, validation_data=(X_val, y_val) ,callbacks=[tensorboard])
#model.save('face.model')

Train on 18 samples, validate on 5 samples
Epoch 1/5
18/18 [==============================] - 20s 1s/step - loss: 0.7018 - acc: 0.7037 - val_loss: 1.1177 - val_acc: 0.6667
Epoch 2/5
18/18 [==============================] - 17s 933ms/step - loss: 0.6442 - acc: 0.6852 - val_loss: 0.4252 - val_acc: 0.7333
Epoch 3/5
18/18 [==============================] - 16s 868ms/step - loss: 0.6217 - acc: 0.7593 - val_loss: 0.2659 - val_acc: 0.9333
Epoch 4/5
18/18 [==============================] - 17s 926ms/step - loss: 0.5276 - acc: 0.7963 - val_loss: 0.4437 - val_acc: 0.7333
Epoch 5/5
18/18 [==============================] - 14s 762ms/step - loss: 0.4903 - acc: 0.7037 - val_loss: 0.7507 - val_acc: 0.6667
Train on 18 samples, validate on 5 samples
Epoch 1/50
18/18 [==============================] - 66s 4s/step - loss: 0.5521 - acc: 0.7222 - val_loss: 0.8478 - val_acc: 0.6667
Epoch 2/50
18/18 [==============================] - 18s 1s/step - loss: 0.4771 - acc: 0.7778 - val_loss: 0.8665 - val_acc: 0.66

KeyboardInterrupt: 